<h1>Image Processing and Handling WS 2018/19</h1>

Exercise instructor: Marko Jovanović, mjovanovic@mi.rwth-aachen.de

<strong style="color: red">Notice: </strong>Attendance to <strong>all</strong> exercises sessions <strong>is mandatory</strong>. However, submitted exercise solutions aren't graded nor they present a prerequiste for the exam, but you will receive feedback on your submitted solutions.

The exercise sessions are held from 12.30-14.00 on the following dates at COMA 1:

22.10.2018 - OpenCV and Python intro (this session)<br />
<strong>05.11.2018 - Image Enhancement</strong><br />
19.11.2018 - Fourier Transform<br />
03.12.2018 - Visualization<br />
17.12.2018 - Automation<br />
14.01.2019 - Low-level Image Segmentation<br />
21.01.2019 - High-level Image Segmentation<br />
28.01.2019 - Solving a Problem<br />

The topics are an orientation and subject to change, in accordance with the lectures.

<h2>Exercise 2: Image Enhancement</h2>

Due date: <strong>12.11.2018</strong>

<h3>List of Tasks</h3>
<ul>
    <li><a href="#Task-1:-A-simplistic-approach-to-contrast-enhancement">Task 1: A simplistic approach to contrast enhancement</a></li>
    <li><a href="#Task-2:-Histogram-Equalization-and-CLAHE">Task 2: Histogram Equalization and CLAHE</a></li>
    <li><a href="#Task-3:-Noise-filtering">Task 3: Noise-filtering</a></li>
</ul>



<h3>Contrast Enhancement</h3>

Contrast can be defined as the difference of luminance and color that makes objects on an image distinguishable. The human eye is more sensitive to contrast than to mere luminance.

Medical images such as X-rays, ultrasound, CT, tomography and others are used in the diagnostic process by physicians. However, often the raw image data may be of lower quality, and poor contrast, which in turn affects the quality of the diagnosis. For example, mammography images may have a low contrast and tissue texture may be shown inappropriately. Since it is often infeasible to repeat the medical imaging for reasons of patient safety (e.g. radiography, CT), image processing techniques for enhancing the image's contrast can be applied in order to accentuate features of interest in the image.
In the following image you can see an original mammogram (Fig. 1a), the same mammogram with applied histogram equalization (Fig. 1b) and with applied contrast-limited adaptive histogram equalization (Fig. 1c). The enhanced images feature a better visualization of important inner structures of the breast.

<img src="mammogram.png" />
<table style="border: none; width: 100%;">
<tr><td style="width: 33%; text-align: center;">(a)</td>
    <td style="width: 33%; text-align: center;">(b)</td>
    <td style="width: 33%; text-align: center;">(c)</td></tr>
</table>
<strong>Figure 1.</strong> Enhancement of structures in the mammogram: (a) original mammogram; (b) mammogram after applying histogram equalizaton; (c) mammogram after applying CLAHE

During these exercise you will learn how to apply contrast enhancement techniques in Python with OpenCV.

<h3 style="color: red">Task 1: A simplistic approach to contrast enhancement</h3>

During the previous exercise, you had been given the task to brighten and darken the image. However, just changing the image's brightness doesn't affect image contrast.

In general, brightness and contrast enhancement is an image processing transform, in which each output pixel's value depends only on the corresponding input pixel value and, potentially, some globally collected image information or parameters.

Multiplication and addition with constants are operations which are commonly used to change the values of individual pixels in the image:

$$ g(i,j) = \alpha f(i,j) + \beta $$

The parameters $\alpha > 0$ and $\beta$ are called <em>gain</em> and <em>bias</em> parameters, which affect the image's contrast and brightness, respectively, where f(i,j) is the input image function, and g(i,j) the output image function.

<strong style="color: red">Programming Task: </strong> Implement this very simple function for contrast adjustment. Try to find the optimal gain and bias parameters, allowing to see the inner structure of the breast more clearly.

<strong>Hint: </strong>
You can use the <code><a href="https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.clip.html">np.clip</a></code> function to clip values to be between 0 and 255.
For your convenience, we have already provided a scaffold of the code.

In [2]:
%matplotlib inline
import cv2
import matplotlib.image as mpimg
import numpy as np
from matplotlib import pyplot as plt

pylab.rcParams['figure.figsize'] = (15.0, 15.0)
img = cv2.imread('mammogram1.png',0)

# Convert to signed 16-bit integer to allow values outside of the (0,255) range
cimg = np.int16(img)  

# YOUR CODE HERE
cimg = img 
# END OF YOUR CODE

# Convert back to uint8 values
cimg = np.uint8(cimg)

plt.subplot(121), plt.imshow(img, cmap = 'gray')
plt.title('Original mammogram'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(cimg, cmap = 'gray')
plt.title('Contrast-adjusted mammogram'), plt.xticks([]), plt.yticks([])
plt.show()

NameError: name 'pylab' is not defined

<h3 style="color: red">Task 2: Histogram Equalization and CLAHE</h3>

<strong>A short reminder about histograms from the last exercise:</strong> A histogram is a graphical representation of a numerical data distribution. To construct a histogram, a range of values is "binned": the entire range of values is usually divided into a series of consecutive and disjoint intervals, and the number of occurences of each value fall into each interval.
An image histogram is a histogram of the pixel intensity values, showing the number of pixels in an image at each different intensity value found in the image. For color RGB images, either 3 histograms for each of the red (R), green (G) and blue (B) channels can be generated, or a 3D-histogram can be produced where each of the axes represent one of the color channels and brightness at each point representing the pixel count.

In this task, you will learn about the concept of histogram equalization and how it can be applied to enhance image contrast.

Consider an image in which pixel values are confined to some specific range of values only: for example, brighter image will have all pixels confined to high values. However, an image of a good quality is supposed to have pixels
from all regions of the image. Therefore, you need to "stretch" its histogram to both ends, which is essentially what histogram equalization performs. This usually improves the contrast of the image. Through this adjustment, the intensities can be better distributed on the histogram. This allows for areas of lower local contrast to gain a higher contrast. Histogram equalization accomplishes this by effectively spreading out the most frequent intensity values.

The method is useful with contrast-limited images with backgrounds and foregrounds that are both bright or both dark and thus may look washed out. In particular, the method can lead to better views of bone structure in x-ray images, and to better detail in photographs that are over or under-exposed.

A key advantage of the method is that it is a fairly straightforward technique and an invertible operator. So, in theory, if the histogram equalization function is known, then the original histogram can be recovered. A disadvantage of the method is that it is indiscriminate and global. It may increase the contrast of background noise, while decreasing the usable signal.

<img src="histogram_equalization.png" />

You can <a href="https://www.math.uci.edu/icamp/courses/math77c/demos/hist_eq.pdf">find here</a> and <a href="http://fourier.eng.hmc.edu/e161/lectures/contrast_transform/node2.html">here</a> a mathematical explanation of histogram equalization.

<strong style="color: red">Programming Task:</strong> Apply histogram equalization on the image mammogram1.png.

<strong>Hint: </strong> In OpenCV, it suffices to use the <code><a href="https://docs.opencv.org/2.4/modules/imgproc/doc/histograms.html?highlight=equalizehist#equalizehist">equalizeHist</a></code> function to apply histogram equalization to an image.


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

pylab.rcParams['figure.figsize'] = (10.0, 10.0)
img = cv2.imread('mammogram1.png',0)

# YOUR CODE HERE
equ = img
# END OF YOUR CODE

plt.subplot(121), plt.imshow(img, cmap = 'gray')
plt.title('Original mammogram'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(equ, cmap = 'gray')
plt.title('Contrast-adjusted mammogram'), plt.xticks([]), plt.yticks([])
plt.show()

<h4>Histogram equalization of color images</h4>
Histogram equalization can also be applied on color images. However, it is not possible to directly apply histogram equalization to the R, G and B components of the image, since this may lead to dramatic changes in the image's color balance because the relative distributions of the color changes would change. Rather, histogram equalization should be applied such that intensity values are equalized without disturbing the color balance of the image. Therefore, we can consider transforming the image from RGB into one of the color space which separates intensity values from color components:

<ul>
    <li>HSV/HLS</li>
    <li>YUV</li>
    <li>YCbCr</li>
</ul>

YCbCr is the preferred color space, as it is designed for digital images. 
Y stands for luminance or luma, obtained from RGB after applied gamma correction. Cr = R-Y (how far the red component is from luma), Cb = B-Y (how far the blue component is from luma). Thus, the luminance and chrominance components are separated into different channels. The color model is mostly used in compression (of the Cr and Cb components) for TV transmission.

Here you can see an example of two images of the same scene, one shot outdoors under bright lighting conditions and one indoors under low lighting conditions, both separated into ints Y,Cr and Cb channels:

<img src="components-ycrcb.png" />

<strong style="color: red">Programming Task: </strong> Load the color image Lenna.png and apply histogram equalization to the Y plane to enhance its contrast. Use the <code><a href="https://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html#cvtcolor">cvtColor</a></code> function to convert the image to the YCrCb color space.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

pylab.rcParams['figure.figsize'] = (15.0, 15.0)
img = cv2.imread('Lenna.png')

# YOUR CODE HERE
equ = img
# END OF YOUR CODE

plt.subplot(121), plt.imshow(img[:,:,::-1])
plt.title('Original Lenna'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(equ[:,:,::-1])
plt.title('Contrast-enhanced Lenna'), plt.xticks([]), plt.yticks([])
plt.show()

However, often this presented simplistic and global approach for histogram equalization doesn't deliver satisfying results, since it considers only the global contrast of the image. Often, images may feature uneven illumination (Fig. 2a). If to such a picture simple histogram equalization is applied, the unevenness of light will be amplified too and the resulting image may as well lose some important structures and details (Fig. 2b).

<img src="cells.png" />
<table style="border: none; width: 100%;">
<tr><td style="width: 33%; text-align: center;">(a)</td>
    <td style="width: 33%; text-align: center;">(b)</td>
    <td style="width: 33%; text-align: center;">(c)</td></tr>
</table>
<strong>Figure 2.</strong> Cell image with uneven illumination: (a) original image; (b) cell image after histogram equalization; (c) cell image after CLAHE.

To alleviate this problem, different adaptive histogram equalization methods can be applied. The methods differ from the simplistic approach in the respect that they compute several histograms over the whole image, each corresponding to a distinct section of the image. The single histograms are used to redistribute the lightness values of the image. Thus, local contrast can be enhanced, and the definition of details in each region of the image is both preserved and enhanced.
Yet, adaptive histogram equalization may still overamplify noise in the image. Contrast Limited Adaptive Histogram Equaliation (CLAHE) tends to prevent this problem by limiting the amplification.

In CLAHE, the image is divided into small blocks ("tiles", each of the size 8x8 by default). Histogram equalization is applied separately on each block. To avoid amplification of noise, contrast limiting is applied. If any histogram bin is above the specified contrast limit (by default 40 in OpenCV), the pixels corresponding to such bins get clipped and distributed uniformly to other bins before applying histogram equalization. To assemble the final image, neighboring tiles are combined using biliear interpolation to eliminate artificially introduced boundaries.

The contrast, especially in homogenous areas, can be limited to avoid amplifying any noise that might be present in the image.

<strong style="color: red">Programming Task: </strong> Apply both simple histogram equalization and CLAHE to the cell.png and mammogram1.png images. Research online for the necessary OpenCV function to use, and adjust the function parameters to achieve best results.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

pylab.rcParams['figure.figsize'] = (15.0, 15.0)
img1 = cv2.imread('cell.png', 0)
img2 = cv2.imread('mammogram1.png', 0)

# YOUR CODE HERE
img1_clahe = img1_equalized = img1
img2_clahe = img2_equalized = img2
# END OF YOUR CODE

plt.subplot(131), plt.imshow(img1, cmap = 'gray')
plt.title('Original image'), plt.xticks([]), plt.yticks([])
plt.subplot(132), plt.imshow(img1_equalized, cmap = 'gray')
plt.title('Histogram Equalized'), plt.xticks([]), plt.yticks([])
plt.subplot(133), plt.imshow(img1_clahe, cmap = 'gray')
plt.title('CLAHE'), plt.xticks([]), plt.yticks([])
plt.show()

plt.subplot(131), plt.imshow(img2, cmap = 'gray')
plt.title('Original image'), plt.xticks([]), plt.yticks([])
plt.subplot(132), plt.imshow(img2_equalized, cmap = 'gray')
plt.title('Histogram Equalized'), plt.xticks([]), plt.yticks([])
plt.subplot(133), plt.imshow(img2_clahe, cmap = 'gray')
plt.title('CLAHE'), plt.xticks([]), plt.yticks([])
plt.show()

<h3 style="color: red">Task 3: Noise filtering</h3>

Biomedical images often suffer from noise and noise reduction techniques have to be applied to reduce the effect of noise on the image and improve the image equality. Two main types of noise are (i) salt and pepper noise, and (ii) Gaussian noise.

<h4>Salt and pepper noise</h4>
Salt and pepper noise is a form of noise sometimes seen on images, caused by sharp and sudden disturbances in the image signal, presenting itself as sparsely occuring black and white pixels (therefore the name). Usually, only a small fraction of the pixels are affected. Salt and pepper noise can effectively be removed with a median filter. Salt and pepper noise may occur from defunct CCD camera sensors or during incorrect signal transmission and analogous to digital conversion.

<img src="screw.png">
<strong>Figure 3.</strong> An image suffering from salt and pepper noise.

<h4>Median filter</h4>
The median filter replaces the central pixel under the filter kernel with the median of all the pixel values under the kernel area. Therefore, no new pixel values are introduced. Since noisy pixels in an image with salt and pepper noise are sparsely introduced, salt and pepper noise can be effectively removed with a median filter.

<strong style="color: red">Programming Task: </strong> Apply a median filter to the screw.png image to remove salt and pepper noise from it. 

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

pylab.rcParams['figure.figsize'] = (10.0, 10.0)
img = cv2.imread('screw.png',0)

# YOUR CODE HERE
img_filtered = img
# END OF YOUR CODE

plt.imshow(img, cmap = 'gray')
plt.subplot(121), plt.imshow(img, cmap = 'gray')
plt.title('Original image'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(img_filtered, cmap = 'gray')
plt.title('Median filtered'), plt.xticks([]), plt.yticks([])
plt.show()

<h4>Gaussian noise</h4>
Unlike salt and pepper noise, Gaussian noise is a statistical noise having a probability density function (PDF) equal to that of the normal distribution (Gaussian distribution). In digital images, Gaussian noise arises during the image acquisition (e.g. from sensor noise caused by poor illumination and/or high temperature), as well as transmission (e.g. electronic circuit noise). For removal of Gaussian noise in an image, a spatial filter can be used. Fig. 3a presents an example of Gaussian noise in an MRI image.

<img src="noise.png" />
<table style="border: none; width: 100%;">
<tr><td style="width: 50%; text-align: center;">(a)</td>
    <td style="width: 50%; text-align: center;">(b)</td></tr>
</table>
<strong>Figure 3.</strong> Gaussian noise in an MRI image: (a) original image; (b) noise removed using non-local means
denoising with custom parameters.

<h4>Averaging filter</h4>
The most simple approach to remove Gaussian noise from an image is to use a normalized block filter (also called averaging filter). The image is convolved with a normalized box filter: the filter takes the average of all pixels under the kernel area and replaces the central element with the average.

Mathematically, a 3x3 normalized box filter takes the following form:

$$ K={1\over 9}\  \begin{bmatrix}
1 & 1 & 1  \\
1 & 1 & 1  \\
1 & 1 & 1
\end{bmatrix} $$

<strong style="color: red">Programming Task: </strong> Apply an averaging filter to the screw.png image.

<strong>Hint: </strong> You can use the <code><a href="https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.ones.html">numpy.ones</a></code> and <code><a href="https://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html">cv2.filter2D</a></code> functions to convolve the filter kernel over the image.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

pylab.rcParams['figure.figsize'] = (10.0, 10.0)
img = mpimg.imread('screw.png')

# YOUR CODE HERE
img_filtered = img
# END OF YOUR CODE

plt.subplot(121), plt.imshow(img),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(img_filtered),plt.title('Average filtered')
plt.xticks([]), plt.yticks([])
plt.show()

<h4>Gaussian blurring</h4>

The Gaussian filter outputs a "weighted average" of each pixel's neighborhood, with the average weighted more towards the value of the central pixels. This is in contrast to the mean filter's uniformly weighted average. Because of this, a Gaussian provides gentler smoothing and preserves edges better than a similarly sized mean filter. 

The Gaussian filter convolves the source image with the specified Gaussian kernel. Here, the idea is to use the Gaussian distribution

$$ G(x,y) = {1 \over 2\pi\sigma^2} e^{-{x^2+y^2 \over 2\sigma^2}}$$

as a "point-spread" function. This can be achieved with convolving the Gaussian filter kernel with the original image. Since the image is a discrete collection of pixel values, we need to work with a discrete approximation of the Gaussian function in order to perform the convolution. The Gaussian distribution has infinite support, meaning it takes non-zero values over its whole domain. Thus, we would theoretically need an infinitely large convolution kernel. However, in practice, the discretized kernel takes effectively zero values at about three standard deviations from the mean, so it gets truncated at that point.

Here is a mathematical representation of the convolution kernel that approximates the Gaussian function with $\sigma = 1$:

$$ K = {1\over 273}\begin{bmatrix}
1 & 4 & 7 & 4 & 1 \\
4 & 16 & 26 & 16 & 4 \\
7 & 26 & 41 & 26 & 7 \\
4 & 16 & 26 & 16 & 4 \\
1 & 4 & 7 & 4 & 1
\end{bmatrix}$$ 

<strong style="color: red;">Programming Task: </strong> Apply a Gaussian blur filter to the images screw.png and mri_noise.png and adjust the parameters as necessary to obtain best results. Find out online which OpenCV function you would have to use. 
How does the applied filtering affect both the noise and the image quality of each image?
In addition, you may also try performing a convolution with a Gaussian filter kernel yourself, as you did in the previous task. Are the obtained results the same?

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

pylab.rcParams['figure.figsize'] = (10.0, 10.0)
img1 = mpimg.imread('screw.png')
img2 = mpimg.imread('mri_noise.png')

# YOUR CODE HERE
img1_filtered = img1
img2_filtered = img2
# END OF YOUR CODE

plt.imshow(img1, cmap = 'gray')
plt.subplot(121), plt.imshow(img1, cmap = 'gray')
plt.title('Original image'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(img1_filtered, cmap = 'gray')
plt.title('Gaussian filtered'), plt.xticks([]), plt.yticks([])
plt.show()
pylab.rcParams['figure.figsize'] = (15.0, 15.0)
plt.imshow(img2, cmap = 'gray')
plt.subplot(121), plt.imshow(img2, cmap = 'gray')
plt.title('Original image'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(img2_filtered, cmap = 'gray')
plt.title('Gaussian filtered'), plt.xticks([]), plt.yticks([])
plt.show()

<strong style="color: red">Programming Task:</strong> In addition, have a look at the <a href="https://docs.opencv.org/3.3.1/d5/d69/tutorial_py_non_local_means.html">Non-Local Means Denoising tutorial</a> from the OpenCV documentation and try applying the techniques presented in the tutorial to the mri_noise.png image. Do you notice any difference?

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

pylab.rcParams['figure.figsize'] = (10.0, 10.0)
img2 = mpimg.imread('mri_noise.png')

# YOUR CODE HERE
img_filtered = img
# END OF YOUR CODE